[Link a la carpeta con todo el TP](https://drive.google.com/drive/u/0/folders/1do-iyf2SzQln-fh8tmu9mSfzLuim5xs5)

# Imports & Download stage

In [ ]:
!pip install category_encoders

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from category_encoders import BinaryEncoder
import pandas as pd
import numpy as np
%matplotlib inline
import pyarrow.parquet as pq
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from scipy.stats import uniform, randint
from sklearn.metrics import f1_score

pd.options.display.float_format = '{:20,.10f}'.format

import gc

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

ACLARACIÓN: Como usé el mismo feature engineering para ambos modelos, lo hice en [otro colab](https://colab.research.google.com/drive/1oolZwdMmiiPSBROS8q7c9taycd5NoJgR#scrollTo=XeHTXwYzXc3o) y me guardé ese dataset (que es el que estoy cargando aquí)

In [ ]:
if not os.path.exists('test.parq'):
  id1="18FlKfUK0ItadAP1jz9KG96KrbWyiQ1zo"
  downloaded1 = drive.CreateFile({'id': id1})
  downloaded1.GetContentFile('test.parq')

if not os.path.exists('train.parq'):
  id2="1H67jcnBBH5eA6elQu95fEDQwWzOuK5Gr"
  downloaded2 = drive.CreateFile({'id': id2})
  downloaded2.GetContentFile('train.parq')

# Init stage

load dataframe

In [ ]:
raw_df = pq.read_table('train.parq').to_pandas()
test_df = pq.read_table('test.parq').to_pandas()

Train/test split based on IPs

In [ ]:
# Create a DataFrame with unique IPs and their corresponding labels
ip_label_df = raw_df[['attacker_ip_enum', 'label']].drop_duplicates()

# Split the unique IPs into two sets with roughly 80%-20% proportions, stratified by label
train_ips, test_ips = train_test_split(ip_label_df['attacker_ip_enum'],
                                       test_size=0.2,
                                       random_state=42,
                                       stratify=ip_label_df['label'])

# Use these sets to filter the original DataFrame
train_df = raw_df[raw_df['attacker_ip_enum'].isin(train_ips)]
val_df = raw_df[raw_df['attacker_ip_enum'].isin(test_ips)]

del raw_df
del ip_label_df
gc.collect()

4

In [ ]:
val_df.head(5)

,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,attack_category,attack_service,attack_time_of_day,attack_day_sin,attack_day_cos,same_country
18,US,ORACLE-BMC-31898,GB,AMAZON-02,http:scan,10,877,0,scan,http,night,0.4338837266,-0.9009688497,0
73,BR,RedeHost Internet Ltda.,IT,Fastweb,http:spam,29,1174,0,spam,http,night,0.4338837266,-0.9009688497,0
74,BR,RedeHost Internet Ltda.,IT,Fastweb,http:bruteforce,29,1174,0,bruteforce,http,night,0.4338837266,-0.9009688497,0
75,PL,Cyber_Folks S.A.,other,TM TECHNOLOGY SERVICES SDN. BHD.,http:bruteforce,30,8170,0,bruteforce,http,night,0.4338837266,-0.9009688497,0
77,BR,RedeHost Internet Ltda.,IT,Fastweb,http:exploit,29,1174,0,exploit,http,night,0.4338837266,-0.9009688497,0


In [ ]:
train_df.head(5)

,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,attack_category,attack_service,attack_time_of_day,attack_day_sin,attack_day_cos,same_country
0,DE,Host Europe GmbH,other,Murat Aktas,http:exploit,0,6466,0,exploit,http,night,0.4338837266,-0.9009688497,0
1,DE,Host Europe GmbH,other,Murat Aktas,http:spam,0,6466,0,spam,http,night,0.4338837266,-0.9009688497,0
2,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0,bruteforce,http,night,0.4338837266,-0.9009688497,1
3,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0,spam,http,night,0.4338837266,-0.9009688497,1
4,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0,exploit,http,night,0.4338837266,-0.9009688497,1


# Encoding stage

Para el encoding traté de mantener la mayor cantidad de información posible de los dataframes. Por eso sólo usé mean cuando había demasiadas categorías.

Además, las categorías mean-encodeadas en el preceptron eran muy importantes, por lo que me pareció que beneficiaría al modelo tenerlas así

Por como me quedó armado el dataset, no tuve que tener ninguna consideración especial por usar un modelo de árboles ya que las únicas columnas que no son un mean/OHE/binary son las del día que están encodeadas con sinoidales

### Drop IP to avoid overfitting/target leak

In [ ]:
train_df.drop(['attacker_ip_enum'], axis=1, inplace=True)
val_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

test_ips = test_df['attacker_ip_enum'] # I'll need these for kaggle submission
test_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

### Countries (binary)

In [ ]:
# Initialize a BinaryEncoder for the 'watcher_country' column
binary_encoder = BinaryEncoder(cols=['watcher_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['watcher_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['watcher_country']])
val_df_encoded = binary_encoder.transform(val_df[['watcher_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'watcher_country' column
train_df.drop('watcher_country', axis=1, inplace=True)
test_df.drop('watcher_country', axis=1, inplace=True)
val_df.drop('watcher_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['watcher_country_0'] = train_df['watcher_country_0'].astype('uint8')
test_df['watcher_country_0'] = test_df['watcher_country_0'].astype('uint8')
val_df['watcher_country_0'] = val_df['watcher_country_0'].astype('uint8')

train_df['watcher_country_1'] = train_df['watcher_country_1'].astype('uint8')
test_df['watcher_country_1'] = test_df['watcher_country_1'].astype('uint8')
val_df['watcher_country_1'] = val_df['watcher_country_1'].astype('uint8')

train_df['watcher_country_2'] = train_df['watcher_country_2'].astype('uint8')
test_df['watcher_country_2'] = test_df['watcher_country_2'].astype('uint8')
val_df['watcher_country_2'] = val_df['watcher_country_2'].astype('uint8')

train_df['watcher_country_3'] = train_df['watcher_country_3'].astype('uint8')
test_df['watcher_country_3'] = test_df['watcher_country_3'].astype('uint8')
val_df['watcher_country_3'] = val_df['watcher_country_3'].astype('uint8')

train_df['watcher_country_4'] = train_df['watcher_country_4'].astype('uint8')
test_df['watcher_country_4'] = test_df['watcher_country_4'].astype('uint8')
val_df['watcher_country_4'] = val_df['watcher_country_4'].astype('uint8')

In [ ]:
# Initialize a BinaryEncoder for the 'attacker_country' column
binary_encoder = BinaryEncoder(cols=['attacker_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attacker_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attacker_country']])
val_df_encoded = binary_encoder.transform(val_df[['attacker_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attacker_country' column
train_df.drop('attacker_country', axis=1, inplace=True)
test_df.drop('attacker_country', axis=1, inplace=True)
val_df.drop('attacker_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attacker_country_0'] = train_df['attacker_country_0'].astype('uint8')
test_df['attacker_country_0'] = test_df['attacker_country_0'].astype('uint8')
val_df['attacker_country_0'] = val_df['attacker_country_0'].astype('uint8')

train_df['attacker_country_1'] = train_df['attacker_country_1'].astype('uint8')
test_df['attacker_country_1'] = test_df['attacker_country_1'].astype('uint8')
val_df['attacker_country_1'] = val_df['attacker_country_1'].astype('uint8')

train_df['attacker_country_2'] = train_df['attacker_country_2'].astype('uint8')
test_df['attacker_country_2'] = test_df['attacker_country_2'].astype('uint8')
val_df['attacker_country_2'] = val_df['attacker_country_2'].astype('uint8')

train_df['attacker_country_3'] = train_df['attacker_country_3'].astype('uint8')
test_df['attacker_country_3'] = test_df['attacker_country_3'].astype('uint8')
val_df['attacker_country_3'] = val_df['attacker_country_3'].astype('uint8')

train_df['attacker_country_4'] = train_df['attacker_country_4'].astype('uint8')
test_df['attacker_country_4'] = test_df['attacker_country_4'].astype('uint8')
val_df['attacker_country_4'] = val_df['attacker_country_4'].astype('uint8')

### AS Names (Mean/Target)

In [ ]:
target_encoder = TargetEncoder(cols=['watcher_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_as_name']])
val_df_encoded = target_encoder.transform(val_df[['watcher_as_name']])

train_df[['watcher_as_name']] = train_df_encoded.astype('float32')
test_df[['watcher_as_name']] = test_df_encoded.astype('float32')
val_df[['watcher_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

In [ ]:
target_encoder = TargetEncoder(cols=['attacker_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['attacker_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['attacker_as_name']])
val_df_encoded = target_encoder.transform(val_df[['attacker_as_name']])

train_df[['attacker_as_name']] = train_df_encoded.astype('float32')
test_df[['attacker_as_name']] = test_df_encoded.astype('float32')
val_df[['attacker_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Watcher UUID Enums (Mean/Target)

In [ ]:
target_encoder = TargetEncoder(cols=['watcher_uuid_enum'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_uuid_enum']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_uuid_enum']])
val_df_encoded = target_encoder.transform(val_df[['watcher_uuid_enum']])

train_df[['watcher_uuid_enum']] = train_df_encoded.astype('float32')
test_df[['watcher_uuid_enum']] = test_df_encoded.astype('float32')
val_df[['watcher_uuid_enum']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Attack type (OHE/Bin)

In [ ]:
#Not sure if I want to use this yet

train_df.drop(['attack_type'], axis=1, inplace=True)
test_df.drop(['attack_type'], axis=1, inplace=True)
val_df.drop(['attack_type'], axis=1, inplace=True)

In [ ]:
# Initialize a BinaryEncoder for the 'attack_service' column
binary_encoder = BinaryEncoder(cols=['attack_service'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attack_service']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attack_service']])
val_df_encoded = binary_encoder.transform(val_df[['attack_service']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attack_service' column
train_df.drop('attack_service', axis=1, inplace=True)
test_df.drop('attack_service', axis=1, inplace=True)
val_df.drop('attack_service', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attack_service_0'] = train_df['attack_service_0'].astype('uint8')
test_df['attack_service_0'] = test_df['attack_service_0'].astype('uint8')
val_df['attack_service_0'] = val_df['attack_service_0'].astype('uint8')

train_df['attack_service_1'] = train_df['attack_service_1'].astype('uint8')
test_df['attack_service_1'] = test_df['attack_service_1'].astype('uint8')
val_df['attack_service_1'] = val_df['attack_service_1'].astype('uint8')

train_df['attack_service_2'] = train_df['attack_service_2'].astype('uint8')
test_df['attack_service_2'] = test_df['attack_service_2'].astype('uint8')
val_df['attack_service_2'] = val_df['attack_service_2'].astype('uint8')

train_df['attack_service_3'] = train_df['attack_service_3'].astype('uint8')
test_df['attack_service_3'] = test_df['attack_service_3'].astype('uint8')
val_df['attack_service_3'] = val_df['attack_service_3'].astype('uint8')

In [ ]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_category' column in both train and test DataFrames
encoded_attack_category_train = encoder.fit_transform(train_df[['attack_category']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_category_train = encoded_attack_category_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_category' column
encoded_attack_category_train_df = pd.DataFrame(encoded_attack_category_train, columns=encoder.get_feature_names_out(['attack_category']))

# Concatenate the one-hot encoded 'attack_category' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_train_df)

# Run GC to save memory
del encoded_attack_category_train, encoded_attack_category_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_category_test = encoder.transform(test_df[['attack_category']])
encoded_attack_category_test = encoded_attack_category_test.todense().astype('int8')
encoded_attack_category_test_df = pd.DataFrame(encoded_attack_category_test, columns=encoder.get_feature_names_out(['attack_category']))
test_df = test_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_test_df)
del encoded_attack_category_test, encoded_attack_category_test_df
gc.collect()

encoded_attack_category_val = encoder.transform(val_df[['attack_category']])
encoded_attack_category_val = encoded_attack_category_val.todense().astype('int8')
encoded_attack_category_val_df = pd.DataFrame(encoded_attack_category_val, columns=encoder.get_feature_names_out(['attack_category']))
val_df = val_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_val_df)
del encoded_attack_category_val, encoded_attack_category_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Attack time of day (OHE)

In [ ]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_time_of_day' column in both train and test DataFrames
encoded_attack_time_of_day_train = encoder.fit_transform(train_df[['attack_time_of_day']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_time_of_day_train = encoded_attack_time_of_day_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_time_of_day' column
encoded_attack_time_of_day_train_df = pd.DataFrame(encoded_attack_time_of_day_train, columns=encoder.get_feature_names_out(['attack_time_of_day']))

# Concatenate the one-hot encoded 'attack_time_of_day' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_train_df)

# Run GC to save memory
del encoded_attack_time_of_day_train, encoded_attack_time_of_day_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_time_of_day_test = encoder.transform(test_df[['attack_time_of_day']])
encoded_attack_time_of_day_test = encoded_attack_time_of_day_test.todense().astype('int8')
encoded_attack_time_of_day_test_df = pd.DataFrame(encoded_attack_time_of_day_test, columns=encoder.get_feature_names_out(['attack_time_of_day']))
test_df = test_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_test_df)
del encoded_attack_time_of_day_test, encoded_attack_time_of_day_test_df
gc.collect()

encoded_attack_time_of_day_val = encoder.transform(val_df[['attack_time_of_day']])
encoded_attack_time_of_day_val = encoded_attack_time_of_day_val.todense().astype('int8')
encoded_attack_time_of_day_val_df = pd.DataFrame(encoded_attack_time_of_day_val, columns=encoder.get_feature_names_out(['attack_time_of_day']))
val_df = val_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_val_df)
del encoded_attack_time_of_day_val, encoded_attack_time_of_day_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Checkpoint!

In [ ]:
train_df.sample(5)

,watcher_as_name,attacker_as_name,watcher_uuid_enum,label,attack_day_sin,attack_day_cos,same_country,watcher_country_0,watcher_country_1,watcher_country_2,...,attack_service_1,attack_service_2,attack_service_3,attack_category_crawl,attack_category_exploit,attack_category_scan,attack_category_spam,attack_category_unknown,attack_time_of_day_day,attack_time_of_day_night
24570277,0.0151618076,0.0002006003,0.0061466722,0,-0.7818315029,0.6234897971,0,0,0,1,...,0,1,1,0,0,0,0,0,1,0
7046031,0.0155606065,0.0018059985,0.0000000000,0,-0.0000000000,1.0000000000,0,0,1,1,...,0,0,1,0,0,1,0,0,0,1
10474880,0.0136674261,0.0000000000,0.0136674261,0,-0.4338837266,-0.9009688497,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
21641029,0.0135492105,0.0000000000,0.0000000011,0,0.9749279022,-0.2225209326,0,0,1,1,...,0,0,1,0,0,1,0,0,0,1
7049693,0.0143511035,0.0000000000,0.0171717163,0,0.9749279022,-0.2225209326,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0


# Model Stage

In [ ]:
# I need all memory I can get
test_df.to_parquet('test_processed.parq')
del test_df
gc.collect()

0

### Hiperparam search

In [ ]:
# Define hyperparameters
n_batches = 5 # ~6M batch size
batch_size=6000000
n_max_iter = 0#1000
cut_score = 0.40

# Split the data into features (X) and target variable (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_batch_size = int(np.floor(X_train.shape[0]/n_batches))
y_batch_size = int(np.floor(y_train.shape[0]/n_batches))

X_test = val_df.drop('label', axis=1)
y_test = val_df['label']

crit_opt = ['gini', 'entropy', 'log_loss']
mf_opt = ['sqrt', 'log2']

# Instantiate the XGBoost model
for n_iter in range(n_max_iter):

  params = {
  'n_estimators': random.randint(10, 24),
  'criterion': crit_opt[random.randint(0, len(crit_opt)-1)],
  'max_depth': random.randint(100, 1000),
  'min_samples_split': random.randint(2, 8),
  'min_samples_leaf': random.randint(1, 5),
  'min_weight_fraction_leaf': 0,
  'max_features': mf_opt[random.randint(0, len(mf_opt)-1)]
  }
  # btw sé que por como lo tengo hecho las opciones son básicamente 10/15/20
  d_estimators = int(params['n_estimators']/(n_batches))
  rf_model = RandomForestClassifier(
      n_estimators = d_estimators,
      criterion=params['criterion'],
      max_depth = params['max_depth'],
      min_samples_split = params['min_samples_split'],
      min_samples_leaf = params['min_samples_leaf'],
      min_weight_fraction_leaf = params['min_weight_fraction_leaf'],
      max_features = params['max_features'],
      warm_start=True,
      random_state = 42
  )

  # Train the model in batches, keeping label proportion
  for i in range(0, n_batches):
    X_batch = X_train.iloc[i*X_batch_size: (i+1)*X_batch_size]
    y_batch = y_train.iloc[i*y_batch_size:i*y_batch_size + y_batch_size]

    rf_model.n_estimators = d_estimators*(i+1)
    rf_model.fit(X_batch, y_batch)

  # Make predictions on the test set
  y_pred = rf_model.predict(X_test)

  # Evaluate the model
  f1_accuracy = f1_score(y_test, y_pred)

  if f1_accuracy > -1:
    max_score = f1_accuracy
    print(f'F1 Accuracy: {f1_accuracy}')
    print(f'Best Params: {params}')


### Reentrega

F1 Accuracy: 0.5822123943257203

Best Params: {'n_estimators': 23, 'criterion': 'log_loss', 'max_depth': 926, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---




## Kaggle submission

### fit final model

In [ ]:
# Define hyperparameters
n_batches = 5 # ~6M batch size
batch_size=6000000
n_max_iter = 0#1000
cut_score = 0.40

# Split the data into features (X) and target variable (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_batch_size = int(np.floor(X_train.shape[0]/n_batches))
y_batch_size = int(np.floor(y_train.shape[0]/n_batches))

X_test = val_df.drop('label', axis=1)
y_test = val_df['label']

params = {
    'n_estimators': 23,
    'criterion': 'log_loss',
    'max_depth': 926,
    'min_samples_split': 2,
    'min_samples_leaf': 5,
    'min_weight_fraction_leaf': 0,
    'max_features': 'sqrt'
  }

d_estimators = int(params['n_estimators']/(n_batches))
rf_model = RandomForestClassifier(
    n_estimators = d_estimators,
    criterion=params['criterion'],
    max_depth = params['max_depth'],
    min_samples_split = params['min_samples_split'],
    min_samples_leaf = params['min_samples_leaf'],
    min_weight_fraction_leaf = params['min_weight_fraction_leaf'],
    max_features = params['max_features'],
    warm_start=True,
    random_state = 42
)

# Train the model in batches, keeping label proportion
for i in range(0, n_batches):
  X_batch = X_train.iloc[i*X_batch_size: (i+1)*X_batch_size]
  y_batch = y_train.iloc[i*y_batch_size:i*y_batch_size + y_batch_size]

  rf_model.n_estimators = d_estimators*(i+1)
  rf_model.fit(X_batch, y_batch)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
f1_accuracy = f1_score(y_test, y_pred)

if f1_accuracy > -1:
  max_score = f1_accuracy
  print(f'F1 Accuracy: {f1_accuracy}')
  print(f'Best Params: {params}')


F1 Accuracy: 0.5822123943257203
Best Params: {'n_estimators': 23, 'criterion': 'log_loss', 'max_depth': 926, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}


### move files around

In [ ]:
test_df = pq.read_table('test_processed.parq').to_pandas()

In [ ]:
train_df.to_parquet('train_encoded.parq')
val_df.to_parquet('val_encoded.parq')
del train_df, val_df
gc.collect()

27

In [ ]:
# in case I need them again
import pyarrow.parquet as pq
train_df = pq.read_table('train_encoded.parq').to_pandas()
val_df = pq.read_table('val_encoded.parq').to_pandas()

FileNotFoundError: ignored

### make submission

In [ ]:
kaggle_pred = rf_model.predict(test_df)
kaggle_df = pd.DataFrame({'attacker_ip_enum': test_ips, 'prediction': kaggle_pred})

In [ ]:
kaggle_df = kaggle_df.groupby('attacker_ip_enum').mean().reset_index()
kaggle_df['prediction'] = kaggle_df['prediction'].round().astype(int)

In [ ]:
kaggle_df.set_index('attacker_ip_enum').to_csv('rf_submission.csv')

# Conclussion stage

### Scores
---
- Validation: 0.58221

- Test (Kaggle): 0.56666

### Reflexiones
---
- Los problemas que tuve en la primera entrega con las demoras de entrenamiento eran problema de escoger hiperparámetros mal (n_estimators sobre todo). Con los estimators bien escogidos y el entrenamiento bien hecho logré conseguir mejores resultados en menos tiempo por fit (como 3min/fit vs 25+min/fit)
---
- En cuanto a la división por batches, intenté hacerlo de la forma correcta (garantizando mantener la proporción de label), y haciendo undersampling, pero ambos me dieron malos resultados (casi todo scores 0). Al final lo que me dió mejores resultados fue la solución "bruta" de dividir el dataset en cachos iguales así como venía.
---
- For the record, comprobé que el split efectivamente fuera el mismo que el de preceptron.